In [6]:
# 1.获取数据
# 2.理解数据内容，进行预处理（数据清洗）
#  - 邮件由两部分组成，第一部分包含了邮件的信息，例如发件人，标题等等，第二部分才是邮件正文。这些文件不是 UTF-8 编码的，所以需要将其转为 UTF-8 编码
#  - 实验会用到的只有邮件正文内容，所以需要去除第一部分，另外正文部分还有许多链接等其他不需要的内容
#    - 转换源数据编码格式为 UTF-8 格式。
#    - 过滤字符：去除所有非中文字符，如标点符号、英文字符、数字、网站链接等特殊字符。
#    - 过滤停用词。
#    - 对邮件内容进行分词处理。
# 2.1.数据清洗
#  - 只保留汉字：通过正则表达式滤掉了所有英文，数字，标点符号，特殊符号，内容里还存在一些长相奇怪的文字，我们通过 Unicode 中文编码范围 0x4e00-0x9fff 过滤
#  - 文本进行分词：利用 结巴分词 模块进行分词
#    - 在中文中，有很多的非实意词语或者其他并没有实际作用的词语，这些词语必须在分词之后进行过滤，这个环节也就是过滤停用词
#    - 分词之后需剔除只有一个字的结果，因为一个字基本上没有什么内容
#  - 文字转向量：文字无法直接被算法理解
# 3.数据划分及建模
#  - 划分训练集和测试集
#  - 

import pandas as pd

# https://cdn.aibydoing.com/hands-on-ai/files/trec06c.zip
data = pd.read_table('./trec06c/full/index', header=None,
                     encoding='gb2312', delim_whitespace=True)
data.head()
# 标记 spam 是垃圾邮件，标记 ham 是正常邮件

C:\Users\91658\AppData\Local\Temp\ipykernel_3860\512922365.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_table('./trec06c/full/index', header=None,


,0,1
0,spam,../data/000/000
1,ham,../data/000/001
2,spam,../data/000/002
3,spam,../data/000/003
4,spam,../data/000/004


In [7]:
df = data.replace(['spam', 'ham'], [0, 1])  # 0 替代 spam，1 替代 ham
df = df.replace(regex=["\.."], value='trec06c')  # 替换掉文件路径
df = df.sample(len(df), random_state=1, )[:10000]  # 打乱样本并取前 1 万条数据
df.groupby(0).count()  # 统计样本

C:\Users\91658\AppData\Local\Temp\ipykernel_3860\78917340.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = data.replace(['spam', 'ham'], [0, 1])  # 0 替代 spam，1 替代 ham


,1
0,
0,6595
1,3405


In [ ]:
# 示例垃圾邮件内容

# Received: from 163.con ([61.141.165.252])
# 	by spam-gw.ccert.edu.cn (MIMEDefang) with ESMTP id j7CHJ2B9028021
# 	for <xing@ccert.edu.cn>; Sun, 14 Aug 2005 10:04:03 +0800 (CST)
# Message-ID: <200508130119.j7CHJ2B9028021@spam-gw.ccert.edu.cn>
# From: =?GB2312?B?1cW6o8TP?= <jian@163.con>
# Subject: =?gb2312?B?uavLvtK1zvEutPq/qreixrGjoQ==?=
# To: xing@ccert.edu.cn
# Content-Type: text/plain;charset="GB2312"
# Date: Sun, 14 Aug 2005 10:17:57 +0800
# X-Priority: 2
# X-Mailer: Microsoft Outlook Express 5.50.4133.2400

# 尊敬的贵公司(财务/经理)负责人您好！  
#         我是深圳金海实业有限公司（广州。东莞）等省市有分公司。  
#     我司有良好的社会关系和实力，因每月进项多出项少现有一部分  
#     发票可优惠对外代开税率较低，增值税发票为5%其它国税.地税.     
#     运输.广告等普通发票为1.5%的税点，还可以根据数目大小来衡  
#     量优惠的多少，希望贵公司.商家等来电商谈欢迎合作。
   
#        本公司郑重承诺所用票据可到税务局验证或抵扣！
#     欢迎来电进一步商谈。
#     电话：13826556538（24小时服务）
#     信箱：szlianfen@163.com
#     联系人：张海南

               
#        顺祝商祺   
                 

#                    深圳市金海实业有限公司

In [8]:
import re

def clean_str(line):
    # 清理邮件，替换不需要的字符串
    line.strip('\n')
    line = re.sub(r"[^\u4e00-\u9fff]", "", line)
    line = re.sub(
        "[0-9a-zA-Z\-\s+\.\!\/_,$%^*\(\)\+(+\"\')]+|[+——！，。？、~@#￥%……&*（）<>\[\]:：★◆【】《》;；=?？]+", "", line)
    return line.strip()

In [13]:
def load_stopwords(file_path):
    # 加载停用词
    with open(file_path, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip('\n') for line in f.readlines()]
    return stopwords

stopwords = load_stopwords('./stopwords.txt')
stopwords[:5]

['!', '"', '#', '$', '%']

In [14]:
import jieba

def process(file_path, test_mode=False):
    # 清洗一封邮件
    '''
    - file_path: 文本文件路径
    - test_mode: 测试模式，后文我们会将一个字符串写入文件(utf-8 编码)，而训练文件以 GBK 编码，
                 如果自己实现分类，请注意编码格式，通常为 utf-8
    - return: words, 处理、分词之后的有效词语
    '''
    words = []
    with open(file_path, 'rb') as f:
        for line in f.readlines():
            if not test_mode:
                line = line.strip().decode("gbk", 'ignore')
            else:
                line = line.strip().decode("utf-8", 'ignore')
            line = clean_str(line)
            if len(line) == 0:
                continue
            seg_list = list(jieba.cut(line, cut_all=False))
            for x in seg_list:
                if len(x) <= 1:
                    continue
                if x in stopwords:
                    continue
                words.append(x)
    return words

In [ ]:
# 进行单个邮件处理
# words = process('trec06c/data/000/000')
# " ".join(words)

In [15]:


from tqdm import tqdm

tqdm.pandas()  # 使用 tqdm 显示进度
# 将 apply 函数替换为 progress_apply 以使用 tqdm 显示处理进度
df["words"] = df[1].progress_apply(process)
df.head()

  0%|                                               | 0/10000 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\91658\AppData\Local\Temp\jieba.cache
Loading model cost 0.654 seconds.
Prefix dict has been built successfully.
100%|██████████████████████████████████| 10000/10000 [01:03<00:00, 158.10it/s]


,0,1,words
37029,1,trec06c/data/123/129,"[恋爱, 第三次, 告诉, 再见面, 时间, 我要, 考研, 考到, 北京, 是否是, 喜欢..."
2257,0,trec06c/data/007/157,"[欣欣, 签约, 推出, 中国, 第一个, 彩铃, 歌手, 稀稀, 龙乐, 公司, 签约, ..."
50881,1,trec06c/data/169/181,"[男生, 思路, 简单, 心痛, 直说, 原因, 不让, 担心, 他累, 不去, 撒娇, 撒..."
10843,0,trec06c/data/036/043,[]
4689,0,trec06c/data/015/189,"[本港, 会计师, 权威机构, 香港, 瑞丰, 会计师, 事务所, 注册, 海外, 国际, ..."


In [21]:
from gensim.models import Word2Vec
from tqdm import tqdm_notebook

# 移除一些不必要的警告
import warnings

warnings.filterwarnings("ignore")

# 导入上面保存的分词数组
data = df["words"]

# 训练 Word2Vec 浅层神经网络模型
w2v_model = Word2Vec(vector_size=100, min_count=10)
w2v_model.build_vocab(data)
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=5)
w2v_model

ImportError: cannot import name 'triu' from 'scipy.linalg' (C:\Users\91658\Miniconda3\envs\ml\Lib\site-packages\scipy\linalg\__init__.py)

In [ ]:
# 将词向量保存为 Ndarray
data_vec = np.concatenate([sum_vec(z) for z in tqdm_notebook(data)])
data_vec.shape

In [ ]:
from sklearn.model_selection import train_test_split

feature_data = data_vec
label_data = df[0].values
# 分割数据
X_train, X_test, y_train, y_test = train_test_split(
    feature_data, label_data, test_size=0.2, random_state=4
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# alpha 表示平滑参数，如拉普拉斯平滑则 alpha=1。

# fit_prior 表示是否使用先验概率，默认为 True。

# class_prior 表示类的先验概率。

# 常用方法:

# fit(x,y)选择合适的贝叶斯分类器。

# predict(X) 对数据集进行预测返回预测结果。

# sklearn.naive_bayes.BernoulliNB(alpha=1.0, binarize=0.0, fit_prior=True, class_prior=None)

In [ ]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()  # 定义伯努利模型分类器
model.fit(X_train, y_train)  # 模型训练
y_pred = model.predict(X_test)  # 模型预测
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)